In [1]:
#create new page rank file for binning
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-246d4563-e782-4fbf-8ab7-fb45a409c8cb;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (33ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (21ms)
:: resolution report :: resolve 665ms :: artifacts dl 57ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/04/09 17:40:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 17:40:15 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
#read everything in
df = readCheckpoint(size='all').cache()

 5:40PM UTC on Apr 09, 2023 --- read and cache time: 8.130888938903809 seconds ---


In [3]:
df.count()

452774474

In [5]:
#Create a new dataframe with only malicious IDs
s3_url_refined = "s3a://sapient-bucket-refined/"
df_labels = spark.read.parquet(f"{s3_url_refined}/prod/labels").cache()
malcious_hosts = list(df_labels.select('hostname').distinct().toPandas()['hostname'])
df_labels.unpersist()

DataFrame[hostname: string, id: string, objectID: string, actorID: string, timestamp: timestamp, object: string, action: string]

In [6]:
df = df.filter(col('hostname').isin(malcious_hosts)).cache()
df.count()

24548160

In [7]:
#now get vertices and edges for the whole graph
from graphframes import GraphFrame

In [8]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)



In [9]:
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df.selectExpr('objectID as id').distinct()
dst_vertices = df.selectExpr('actorID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df.selectExpr('objectID as src', 'actorID as dst', 'id', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user')

# Create GraphFrame
g = GraphFrame(vertices, edges)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [10]:
src_vertices.unpersist()
dst_vertices.unpersist()
vertices.unpersist()
edges.unpersist()

DataFrame[src: string, dst: string, id: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string]

In [11]:
df.unpersist()
start_time = time.time()
pagerank = g.pageRank(resetProbability=0.15, maxIter=10)
g.unpersist()
pagerank_v = pagerank.vertices
pagerank_v.limit(10).show()
print(time.time() - start_time)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------------------+-------------------+
|                  id|           pagerank|
+--------------------+-------------------+
|000b6bbf-fddd-456...|0.18349002444011722|
|009219d6-7a35-4b3...|0.18349002444011722|
|00c9b78d-0976-407...|0.18349002444011722|
|00d3180f-06c7-4aa...| 0.2402051229034262|
|0174d4d5-8c21-42f...|0.18349002444011722|
|01c8b545-a730-401...|0.18349002444011722|
|059e5047-169c-4ae...|0.18349002444011722|
|064453d1-8e7f-436...| 0.4143589938381115|
|06ca5760-85df-471...| 0.7903382916247628|
|075e0c5d-bf61-4a8...|0.18349002444011722|
+--------------------+-------------------+

994.8354897499084


In [12]:
#now write pagerank into s3
s3_url_trusted = "s3a://sapient-bucket-trusted/"
pagerank_v.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/pagerank")

In [13]:
#unpersist and read in new pagerank df
pagerank_v.unpersist()

DataFrame[id: string, pagerank: double]

In [14]:
start_time = time.time()
pr = spark.read.parquet(f"{s3_url_trusted}/prod/graph/pagerank").cache()
pr.count()
print("read and count: " + str(time.time() - start_time))

read and count: 0.8624114990234375


In [13]:
#caunpersistlate quartiles and see what we have
bins = pr.approxQuantile("pagerank", [.25, .5, .75], 0)
bins = [float("-inf")] + bins + [float("inf")]
print(bins)

[-inf, 0.1834900244401172, 0.1834900244401172, 0.1834900244401172, 0.1834900244401172, 0.1834900244401172, 0.1834900244401172, 0.1834900244401172, inf]


In [16]:
pr.limit(10).show()

+--------------------+-------------------+
|                  id|           pagerank|
+--------------------+-------------------+
|00aaa800-0086-439...|0.18349002444011722|
|00d4c7c3-8d68-406...|0.18349002444011722|
|0113087d-3933-469...|0.18349002444011722|
|02181d48-7f1b-493...|0.18349002444011722|
|02945258-0806-4e6...|0.18349002444011722|
|03643915-1c95-4ea...|0.18349002444011722|
|0398e693-85a9-4ba...|0.24371709630826954|
|0499b8d9-f325-4fa...|0.18349002444011722|
|04b33e03-c5c3-483...|0.18349002444011722|
|0718e14e-e693-410...|0.18349002444011722|
+--------------------+-------------------+



In [18]:
start_time = time.time()
print(start_time)

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import numpy as np

pagerank_values = pr.select("pagerank").rdd.flatMap(lambda x: x).collect()
sorted_pagerank_values = sorted(pagerank_values)

Q1 = np.percentile(sorted_pagerank_values, 25)
Q2 = np.percentile(sorted_pagerank_values, 50)
Q3 = np.percentile(sorted_pagerank_values, 75)

print(Q1)
print(Q2)
print(Q3)

def assign_quartile(score):
    if score <= Q1:
        return 'Q1'
    elif score <= Q2:
        return 'Q2'
    elif score <= Q3:
        return 'Q3'
    else:
        return 'Q4'

quartile_udf = udf(assign_quartile, StringType())

pagerank_with_quartiles = pr.withColumn("quartile", quartile_udf(col("pagerank")))

print("binning: " + str(time.time() - start_time))

1681063828.199317
0.18349002444011722
0.18349002444011722
0.18349002444011722
binning: 1.3553645610809326


In [23]:
pr.count()

1832752

In [21]:
pr.where(col('pagerank') == .18349002444011722).count()

1624066

In [22]:
pr.where(col('pagerank') < .18349002444011722).count()

0

In [24]:
pr.where(col('pagerank') > .18349002444011722).count()

208686

In [15]:
pr.sort(col('pagerank').desc()).limit(10).show()

+--------------------+------------------+
|                  id|          pagerank|
+--------------------+------------------+
|48ff772b-9d7a-4d6...| 9766.130483955527|
|69f43ba8-b058-4eb...| 9056.849296313381|
|8292f41b-a538-4c7...| 8946.121567611614|
|f52f80c5-d127-429...| 8391.015302574988|
|b81dc0d4-e3d1-47b...| 8330.554844677174|
|cc3a1844-0de0-4d5...| 7776.105202023155|
|e718fffa-fe6d-4b4...| 7148.167737517007|
|201719d5-c2ed-43c...|6797.8294521228045|
|40ea1a4c-4ebb-43f...| 6715.257243866726|
|18e861bc-8916-4db...| 6466.098883578839|
+--------------------+------------------+



In [18]:
pr.select(col('pagerank') > .1834900244401172).count()

1832752

In [19]:
pr.select(col('pagerank') < .1834900244401172).count()

1832752

In [25]:
pr.unpersist()

DataFrame[id: string, pagerank: double]

In [18]:
from pyspark.ml.feature import Bucketizer
# if timestamp_bins exists then drop the column

# Create a Bucketizer to group the PageRank scores into 10 bins
bucketizer = Bucketizer(splits=bins, inputCol="pagerank", outputCol="bins")

# Apply the Bucketizer to the DataFrame and add the new column
pr = bucketizer.transform(pr)

# Show the results
pagerank_vertices.limit(10).show()
#returns error as I suspected

IllegalArgumentException: Bucketizer_7041aff0b029 parameter splits given invalid value [-Infinity,0.1834900244401172,0.1834900244401172,0.1834900244401172,0.1834900244401172,Infinity].

### Run everythiing all over again with all the values

In [26]:
#pr.unpersist()
#run it all over again with everything and see what you get
start_time = time.time()
print(start_time)
df = readCheckpoint(size='all').cache()

1681064110.1230302
 6:15PM UTC on Apr 09, 2023 --- read and cache time: 0.40561342239379883 seconds ---


In [27]:
print(df.count())
print('read and count: ' +str(time.time()-start_time))

452774474
read and count: 4.671190977096558


In [28]:
#now get vertices and edges for the whole graph
from graphframes import GraphFrame
start_time = time.time()
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df.selectExpr('objectID as id').distinct()
dst_vertices = df.selectExpr('actorID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df.selectExpr('objectID as src', 'actorID as dst', 'id', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                      'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user')

# Create GraphFrame
g = GraphFrame(vertices, edges)

src_vertices.unpersist()
dst_vertices.unpersist()
vertices.unpersist()
edges.unpersist()

print('graph build time: ' +str(time.time()-start_time))

graph build time: 0.05776548385620117


In [29]:
df.unpersist()
start_time = time.time()
print(start_time)
pagerank = g.pageRank(resetProbability=0.15, maxIter=10)
pagerank_v = pagerank.vertices
pagerank_v.limit(10).show()
g.unpersist()
print("pagerank time: " + str(time.time() - start_time))

1681064152.966351


23/04/09 18:35:09 ERROR Executor: Exception in task 6.0 in stage 724.0 (TID 17268)
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.lang.Long.valueOf(Long.java:840)
	at scala.runtime.BoxesRunTime.boxToLong(BoxesRunTime.java:71)
	at scala.runtime.ScalaRunTime$.array_apply(ScalaRunTime.scala:61)
	at org.apache.spark.util.collection.OpenHashSet.rehash(OpenHashSet.scala:242)
	at org.apache.spark.util.collection.OpenHashSet.rehashIfNeeded(OpenHashSet.scala:171)
	at org.apache.spark.util.collection.OpenHashSet.rehashIfNeeded$mcJ$sp(OpenHashSet.scala:169)
	at org.apache.spark.graphx.util.collection.GraphXPrimitiveKeyOpenHashMap$mcJI$sp.update$mcJI$sp(GraphXPrimitiveKeyOpenHashMap.scala:77)
	at org.apache.spark.graphx.impl.EdgePartitionBuilder.toEdgePartition(EdgePartitionBuilder.scala:65)
	at org.apache.spark.graphx.EdgeRDD$.$anonfun$fromEdges$1(EdgeRDD.scala:110)
	at org.apache.spark.graphx.EdgeRDD$.$anonfun$fromEdges$1$adapted(EdgeRDD.scala:105)
	at org.apache.spark.graphx.Ed

Py4JJavaError: An error occurred while calling o291.run.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 724.0 failed 1 times, most recent failure: Lost task 6.0 in stage 724.0 (TID 17268) (ip-172-16-74-163.us-west-2.compute.internal executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.lang.Long.valueOf(Long.java:840)
	at scala.runtime.BoxesRunTime.boxToLong(BoxesRunTime.java:71)
	at scala.runtime.ScalaRunTime$.array_apply(ScalaRunTime.scala:61)
	at org.apache.spark.util.collection.OpenHashSet.rehash(OpenHashSet.scala:242)
	at org.apache.spark.util.collection.OpenHashSet.rehashIfNeeded(OpenHashSet.scala:171)
	at org.apache.spark.util.collection.OpenHashSet.rehashIfNeeded$mcJ$sp(OpenHashSet.scala:169)
	at org.apache.spark.graphx.util.collection.GraphXPrimitiveKeyOpenHashMap$mcJI$sp.update$mcJI$sp(GraphXPrimitiveKeyOpenHashMap.scala:77)
	at org.apache.spark.graphx.impl.EdgePartitionBuilder.toEdgePartition(EdgePartitionBuil

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 38570)
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/accumulators.

In [ ]:
#write, read, and count (and cache)
start_time = time.time()
s3_url_trusted = "s3a://sapient-bucket-trusted/"
pagerank_v.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/pagerank2")
pagerank_v.unpersist()

In [ ]:
pr = spark.read.parquet(f"{s3_url_trusted}/prod/graph/pagerank").cache()
pr.count()
print("read, write, count time: " + str(time.time() - start_time))